In [1]:
import pandas as pd
#import numpy as np
import gurobipy as gp

In [2]:
distance = pd.read_csv('D:/python/d32f0742246d11ee/dataset/Distance_Matrix.csv')
forecast_2018 = pd.read_csv('D:/python/d32f0742246d11ee/dataset/pred_2018.csv')
forecast_2019 = pd.read_csv('D:/python/d32f0742246d11ee/dataset/pred_2019.csv')

In [3]:
## reorganizing
distance = distance.drop(columns='Unnamed: 0')
forecast_2018 = forecast_2018.drop(columns='Unnamed: 0')
forecast_2019 = forecast_2019.drop(columns='Unnamed: 0')
forecast_2018.set_index('Index',drop=True,inplace=True)
forecast_2019.set_index('Index',drop=True,inplace=True)

In [4]:
cluster_indices_0 = [1989, 1990, 1991, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2153, 2154, 2181, 2182, 2183, 2184, 2185, 2186, 2187, 2188, 2189, 2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2235, 2258, 2259, 2260, 2261, 2262, 2263, 2264, 2265, 2266, 2267, 2268, 2269, 2270, 2271, 2272, 2273, 2290, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2317, 2318, 2319, 2320, 2321, 2322, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 2339, 2340, 2341, 2342, 2343, 2344, 2345, 2346, 2347, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2355, 2356, 2357, 2358, 2359, 2360, 2361, 2362, 2363, 2364, 2365, 2366, 2367, 2368, 2369, 2370, 2371, 2372, 2373, 2374, 2375, 2376, 2377, 2378, 2379, 2380, 2381, 2382, 2383, 2384, 2385, 2386, 2387, 2388, 2389, 2390, 2391, 2392, 2393, 2394, 2395, 2396, 2397, 2398, 2399, 2400, 2401, 2402, 2403, 2404, 2405, 2406, 2407, 2408, 2409, 2410, 2411, 2412, 2413, 2414, 2415, 2416, 2417]
cluster_indices_1 = [196, 197, 198, 199, 200, 201, 202, 203, 204, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 321, 322, 323, 324, 325, 326, 327, 328, 366, 367, 368, 369, 370, 371, 372, 373, 374, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1199, 1200, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1396, 1397]

In [5]:
cluster_indices = [cluster_indices_0,cluster_indices_1]

In [6]:
a = 0.001
b = 1
c = 1
Cap_Depot = 20000
Cap_Refinary = 100000
location = {}

In [7]:
def optimus_prime(Cluster_indices, forecast):
    farm = Cluster_indices
    farm_depot = Cluster_indices
    depot_refinary = Cluster_indices
    refinary = Cluster_indices
    
    farm_depot_keys = [(f,d) for f in farm for d in farm_depot]
    depot_refinary_keys = [(d,r) for d in depot_refinary for r in refinary ]
    dist_dict = {(f,d): distance.loc[f,str(d)] for f in farm for d in farm_depot}
    # Define the Gurobi model
    model = gp.Model('optimize')
    # Define the variables
    open_depot = { d : model.addVar(vtype=gp.GRB.BINARY, name=f'open_depot_{d}')  for d in farm_depot}
    open_refinary = { d : model.addVar(vtype=gp.GRB.BINARY, name=f'open_refinary_{d}') for d in depot_refinary}
    biomass_flows = {(f, d): model.addVar(lb=0, vtype=gp.GRB.CONTINUOUS, name=f'biomass_flows_{f}_{d}') for (f, d) in farm_depot_keys}
    pellet_flows = {(d, r): model.addVar(lb=0, vtype=gp.GRB.CONTINUOUS, name=f'pellet_flows_{d}_{r}') for (d, r) in depot_refinary_keys}
    # Set the objective function
    Transport_cost_biomass = gp.quicksum(a * dist_dict.get((f, d)) * biomass_flows.get((f, d)) for f in farm for d in farm_depot )
    Transport_cost_pellet = gp.quicksum(a * dist_dict.get((d,r)) * pellet_flows.get((d, r)) for d in depot_refinary for r in refinary)
    Underutilization_cost_biomass = gp.quicksum(open_depot.get(d) * Cap_Depot for d in farm_depot) - gp.quicksum(biomass_flows.get((f, d)) for f in farm for d in farm_depot) 
    Underutilization_cost_pellet = gp.quicksum(open_refinary.get(d) * Cap_Refinary for d in depot_refinary) - gp.quicksum(pellet_flows.get((d, r)) for d in depot_refinary for r in refinary)
    cost_forecast = sum(forecast['mae'])
    model.setObjective(Transport_cost_biomass + Transport_cost_pellet + cost_forecast +Underutilization_cost_biomass+Underutilization_cost_pellet , gp.GRB.MINIMIZE)
    # Add constraints
    for f in farm:
        for d in farm_depot:
            model.addConstr(biomass_flows.get((f,d)) >= 0 ,name =f'bio_flow_{f}_{d}')
    for d in depot_refinary:
        for r in refinary:
            model.addConstr(pellet_flows.get((d,r)) >= 0 , name = f'pel_flow_{d}_{r}')
    for d in farm_depot:
        model.addConstr(gp.quicksum(biomass_flows.get((f, d)) for f in farm) <=  Cap_Depot, name=f'depot_{d}_limit')
    for r in refinary:
        model.addConstr(gp.quicksum( pellet_flows.get((d, r)) for d in depot_refinary) <=  Cap_Refinary, name=f'refinery_{r}_limit')
    model.addConstr(gp.quicksum(open_depot.get(d) for d in farm_depot) >= 3, name='depot_limit_lower')
    model.addConstr(gp.quicksum(open_refinary.get(d) for d in depot_refinary) >= 1, name='refinery_limit_lower')
    model.addConstr(gp.quicksum(open_depot.get(d) for d in farm_depot) <= 7, name='depot_limit_upper')
    model.addConstr(gp.quicksum(open_refinary.get(d) for d in depot_refinary) <= 3, name='refinery_limit_upper')
    for f in farm:
        model.addConstr(gp.quicksum(open_depot.get(d) * biomass_flows.get((f, d)) for d in farm_depot) >= 0.8 * forecast['forecast'].loc[f], name=f'farm_{f}_lower_limit')
    for f in farm:
        model.addConstr(gp.quicksum(biomass_flows.get((f, d)) for d in farm_depot) <= forecast['forecast'].loc[f], name=f'farm_{f}_upper_limit')
    for f in farm:
        for d in farm_depot:
            model.addConstr(biomass_flows.get((f, d)) <= open_depot.get(d) * forecast['forecast'].loc[f], name=f'link_{f}_{d}_bio')
    for d in farm_depot:
        model.addConstr(gp.quicksum(biomass_flows[i, d] for i in farm) == gp.quicksum(open_refinary.get(r)*pellet_flows[d,r] for r in refinary), name=f'balance_depot_{d}')        
    
    model.addConstr(gp.quicksum( biomass_flows.get((f, d)) for f in farm for d in farm_depot) == gp.quicksum( pellet_flows.get((d, r))for d in depot_refinary for r in refinary), name=f'balance_{d}')
    for d in farm_depot:
        model.addConstr(open_depot.get(d) + open_refinary.get(d) <= 1, name=f'one_depot_{d}')
    
    model.setParam("MIPGap", 1e-3)
    model.setParam("MIPFocus", 2)
    model.setParam("Threads", 8) 
    # Optimize the model
    model.optimize()
    
    open_depot_values = {var.varName: var.x for var in open_depot.values()}
    open_refinary_values = {var.varName: var.x for var in open_refinary.values()}
    
    location.append(open_depot_values)
    location.append(open_refinary_values)    
    
    
    return location

In [8]:
for c in cluster_indices:
    optimus_prime(Cluster_indices = c, forecast = forecast_2018)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-14


TypeError: can't multiply sequence by non-int of type 'float'